In [ ]:
import os

In [ ]:
working_dir = os.path.dirname(os.getcwd())
os.chdir(working_dir)

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [23]:
readpath = './data/'

be = pd.read_csv(readpath + 'featureset.csv')
# count distinct dates for each user
userlist = be['pid'].unique()

In [24]:
be.rename(columns={'pid': 'who'}, inplace=True)

In [27]:
be = be.assign(t_start = pd.to_datetime(be['t_start']),
          t_end = pd.to_datetime(be['t_end']))

# Chain Calculation for each user

In [36]:
be['date'] = be.t_start.dt.date

In [40]:
be = be.groupby(['who', 'date']).apply(lambda df: df.sort_values('t_start'))
be['seiqd'] = be.groupby(['who', 'date']).cumcount()

In [68]:
columns = ['who','date', 'seiqd', 't_start', 't_end', 'poi_id', 'longitude', 'latitude', 'X', 'Y',
       'ptype', 'fnid', 'chess_x', 'chess_y', 'LU_Business', 'LU_City_Road',
       'LU_Consumption', 'LU_Culture', 'LU_Industry', 'LU_Medical',
       'LU_Park_&_Scenery', 'LU_Public', 'LU_Residence',
       'LU_Science_&_Education', 'LU_Special', 'LU_Transportation', 'LU_Wild']

be = be.reindex(columns=columns)
# convert the date to integer as %Y%m%d format
be['date'] = be['date'].apply(lambda x: int(x.strftime('%Y%m%d')))

In [70]:
be.to_csv(readpath + 'featureset.csv', index=False)  

In [58]:
# deprecated: below
def act2travel(act_data):
    data = {
        'seiqd': act_data['seiqd'][1:].values,
        'poi_o': act_data['poi_id'][:-1].values,
        'poi_d': act_data['poi_id'][1:].values,
        'org_chess_x': act_data['chess_x'][:-1].values,
        'org_chess_y': act_data['chess_y'][:-1].values,
        'des_chess_x': act_data['chess_x'][1:].values,
        'des_chess_y': act_data['chess_y'][1:].values,
        'lambda_o': act_data['longitude'][:-1].values,
        'phi_o': act_data['latitude'][:-1].values,
        'lambda_d': act_data['longitude'][1:].values,
        'phi_d': act_data['latitude'][1:].values,
    }
    lu_attr = act_data.loc[:,act_data.columns.str.startswith('LU_')]
    lu_attr = lu_attr[1:]
    res = pd.concat([pd.DataFrame(data=data).reset_index(drop=True), lu_attr.reset_index(drop=True)], axis=1)
    return res

be_grouped = be.groupby(['who', 'date'])
be_group = be_grouped.apply(act2travel)
be_group